<a href="https://colab.research.google.com/github/golnoushfarzan/Thesis/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import numpy as np
#this one is used to compute binomial coefficients (for C_3)
from scipy.special import comb
#This one is used for numerical integration of a given function over a specified interval (for I term)
from scipy.integrate import quad
import mpmath
from functools import partial # This one is used to create a version of a function with some arguments fixed(for optimization)
from scipy.optimize import minimize
import pandas as pd

#This is the computation for C3 on the half line:

In [92]:
# Define C3 function as in equation 2.25. This one is for \sigma_1

# Define b1(X_0) based on the provided formula in equation 3.11
def b1(X_0):
    return (6 / np.pi**2) + (0.1333 / np.sqrt(X_0))

# Define m_0 based on the provided formula in equation 2.82
def m0():
    return np.sqrt(1 + (2 / 3) * np.sqrt(6 / 5))

# Define b2(X_0) as a constant value in eq 3.15
def b2(X_0):
    return 1.048  # Given value for b2(X_0) for X_0 \geq 1000

# Define D1 and D2 as given in equation 4.25
def D1(X):
    return 6 / np.pi**2 + b2(X) / np.log(X)

def D2(X):
    return (np.pi * m0() * b1(X) / np.log(X)) + (6 * m0() / (np.pi * X)) + (np.pi * m0() * b2(X) / (X * np.log(X)))

# Define the integral I_alpha_beta as in equation \eqref{eq:2.23} (2.25)
def I_alpha_beta(A, n, alpha, beta):
    def integrand(x):
        return x**A * np.exp(-alpha * x**beta) * (np.log(x)**n)

    result, _ = quad(integrand, 0, np.inf)
    return result

# Define constants(Example value)
X = 3 * 1e12
T2 = 3 * 1e12
alpha = 0.241
beta = 2

j_1 = 0.618
j_3 = float(mpmath.zeta(1/2)**2)

a = 4/3
b = 2
# List of values for a_i, b_i, and d_i based on the table of 4.1 (for sigma_1)
a_i_values = [0, -1/6, -1/3, -1/3, -1/2, -2/3, -1, -7/6, -4/3, -4/3, -3/2, -5/3]
b_i_values = [0, 0, 0, -2, -2, -2, 0, 0, 0, -2, -2, -2]

def compute_d_i(X, D1, D2):
    """
    Calculate the d_i values based on the provided formula.
    """

    d_i_values = [1,
                  2 / np.sqrt(D1(X) * np.log(X)),
                  1 / (D1(X) * np.log(X)),
                  j_3 / j_1**2,
                  j_3 / (j_1**2 * np.sqrt(D1(X) * np.log(X))),
                  j_3 / (2 * j_1**2 * D1(X) * np.log(X)),
                  (X * D2(X)) / D1(X),
                  2 * X * D2(X) / (D1(X) * np.sqrt(D1(X) * np.log(X))),
                  (X * D2(X)) / (D1(X)**2 * np.log(X)),
                  j_3 * D2(X) * X / (2 * j_1**2 * D1(X)),
                  j_3 * D2(X) * X / (j_1**2 * D1(X) * np.sqrt(D1(X) * np.log(X))),
                  j_3 * D2(X) * X / (j_1**2 * D1(X)**2 * np.log(X))]
    return d_i_values


# Define the d_i values based on the formula
#def compute_d_i(i, X, D1, D2, j_1, j_3):
    """
    Calculate the d_i values based on the provided formula.
    """
"""
    D1_value = D1(X)
    D2_value = D2(X)

    if i == 0:
        return 1
    elif i == 1:
        return 2 / np.sqrt(D1(X) * np.log(X))
    elif i == 2:
        return 1 / (D1(X) * np.log(X))
    elif i == 3:
        return j_3 / j_1**2
    elif i == 4:
        return j_3 / (j_1**2 * np.sqrt(D1(X) * np.log(X)))
    elif i == 5:
        return j_3 / (2 * j_1**2 * D1(X) * np.log(X))
    elif i == 6:
        return (X * D2(X)) / D1(X)
    elif i == 7:
        return 2 * X * D2(X) / (D1(X) * np.sqrt(D1(X) * np.log(X)))
    elif i == 8:
        return (X * D2(X)) / (D1(X)**2 * np.log(X))
    elif i == 9:
        return j_3 * D2(X) * X / (2 * j_1**2 * D1(X))
    elif i == 10:
        return j_3 * D2(X) * X / (j_1**2 * D1(X) * np.sqrt(D1(X) * np.log(X)))
    elif i == 11:
        return j_3 * D2(X) * X / (j_1**2 * D1(X)**2 * np.log(X))
"""



# Define C3 function as equation 2.25. This one is for \sigma_1
def C3(T2, alpha, beta, a, a_i_values, b, b_i_values,  d_i_values, printing=0):
    log_T = np.log(T2)

    C31_value = 0

    for i in range(len(a_i_values)):
        a_i = a_i_values[i]
        b_i = b_i_values[i]
        d_i = d_i_values[i]
        # Calculate d_i using the function for d_i

        inner_sum = 0
        # Summation over k
        for k in range(b + b_i + 1):
            binomial_coeff = comb(b + b_i, k)
            integral_value = I_alpha_beta(beta + a + a_i - 1, b + b_i - k, alpha, beta)
            inner_sum += binomial_coeff * integral_value / (log_T**(b + b_i - k))

        # definition of C3
        C31_value += d_i * T2**a_i * (log_T**b_i) * inner_sum

    if printing > 0:
        print(f"C3_value is {C31_value}")
    return C31_value

C31_value = C3(T2, alpha, beta, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2))
print(f"The value of C_3 on the half line is: {C31_value}")

The value of C_3 on the half line is: 5.473783347188228


In [93]:
# Define D4 as equation 4.54
j_1 = 0.618
def D4(X):
    return D1(X) * (j_1 ** 2)

D4_value = D4(X)
print(f"The value of D_4 is: {D4_value}")

The value of D_4 is: 0.24611377987225733


In [94]:
log_X = np.log(X)
print(f"The value of log_X is: {log_X}")

The value of log_X is: 28.729633404596658


In [95]:
C31_times_D4 = C31_value * D4_value
print(f"The value of C3 . D4 on the half line is: {C31_times_D4}")

The value of C3 . D4 on the half line is: 1.3471735097783113


#FIRST EXPONENT

In [96]:
delta = 0.36
sigma_2 = 1 + (delta / log_X)
sigma_1 = 0.5
delta_2 = 0.35
sigma = 0.9
sigma_prime = sigma - (delta_2 / np.log(T2))
exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
print(f"The value of exponent is: {exponent}")

The value of exponent is: 0.24332821337602245


#This is the final result on the half line (A1^exponent):

In [97]:
result1 = (C31_times_D4) ** exponent
print(f"The result for the half line: {result1}")

The result for the half line: 1.0752077780603713


#This is the computation for C3 on the sigme2:

In [98]:
# Define C3 function as in equation 2.25. This one is for \sigma_2
gamma = mpmath.euler
# Define b_4 function based on the given equation 3.40
def b_4(X_0):
    return 0.428 + 0.895 / np.log(X_0)

# Define b3(X_0) as a constant value in eq 3.21
def b_3(X_0):
    return 0.605  # Given value for b2(X_0) for X_0 \geq 1000

# Define m_0 based on the provided formula in equation 2.82
def m0():
    return np.sqrt(1 + (2 / 3) * np.sqrt(6 / 5))

# Define D_5 and D_6 based on the given equations
def D5(delta, X):
    """
    Calculate D_5 for given delta and X, using m0 and gamma.
    """
    b4_X = b_4(X)
    return (np.pi * m0() * b4_X) / (2 * delta) * np.exp((2 * delta * float(gamma) / np.log(X)))

def D6(delta, X):
    """
    Calculate D_6 for given delta and X, using gamma.
    """
    b3_X = b_3(X)
    b4_X = b_4(X)
    return (b4_X / (5 * delta * np.exp(float(gamma)))) + (b3_X * np.exp(-2 * delta)) / (np.log(X)**2)


# Define constants(Example value)
X = 3 * 1e12
T2 = 3 * 1e12
alpha = 0.241
beta = 2
a_prime = 1
b_prime = 0

# List of values for a_i', b_i', and d_i' based on the table of 4.3 (for sigma_2)
a_i_prime_values = [0, -1, -1]
b_i_prime_values = [0, 0, 0]

# Define the d_i values based on your formula
def compute_d_i_second(X, D5, D6, delta):
    """
    Calculate the d_i values based on the provided formula.
    """
    D5_value = D5(delta, X)
    D6_value = D6(delta, X)

    d_i_values = [1, (D5(delta, X) * X ) / D6(delta, X), np.pi * m0()]
    return d_i_values


C32_value = C3(T2, alpha, beta, a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta))

print(f"The value of C_3 at sigma_2 is: {C32_value}")

The value of C_3 at sigma_2 is: 42.38082568387099


In [99]:
# Define D6
def D6(delta, X):
    """
    Calculate D_6 for given delta and X, using gamma.
    """
    b3_X = b_3(X)
    b4_X = b_4(X)
    return (b4_X / (5 * delta * np.exp(float(gamma)))) + (b3_X * np.exp(-2 * delta)) / (np.log(X)**2)
D6_value = D6(delta, X)
print(f"The value of D_6 is: {D6_value}")

The value of D_6 is: 0.14357651963994592


In [100]:
C32_times_D6 = C32_value * D6_value
print(f"The value of C3 . D6 at sigma_2 is: {C32_times_D6}")

The value of C3 . D6 at sigma_2 is: 6.084891451157427


#SECOND EXPONENT

In [101]:
delta = 0.36
sigma_2 = 1 + (delta / np.log(X))
sigma_1 = 0.5
delta_2 = 0.35
sigma = 0.9
sigma_prime = sigma - (delta_2 / np.log(T2))
exponent2 = (sigma_prime - sigma_1) / (sigma_2 - sigma_1)
print(f"The value of exponent2 is: {exponent2}")

The value of exponent2 is: 0.7566717866239775


#This is the final result on sigma2 (A2^exponent2):

In [102]:
result2 = (C32_times_D6) ** exponent2
print(f"The result for sigma2: {result2}")

The result for sigma2: 3.9212270870277846


#This is the definition for \tilde{C_5} as equation 4.85.

In [103]:
def tilde_C5(sigma, alpha, beta, delta, delta_2, a, a_i, b, b_i, d_i,
              a_prime, a_i_prime, b_prime, b_i_prime, d_i_prime, T2, printing=0):
    X = T2

    sigma_2 = 1 + (delta / np.log(X))
    sigma_1 = 0.5
    sigma_prime = sigma - (delta_2 / np.log(T2))
    exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
    exponent2 = (sigma_prime - sigma_1) / (sigma_2 - sigma_1)
    D4_value = D4(X)
    C31_value = C3(T2, alpha, beta, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2), printing-1)
    C31_times_D4 = C31_value * D4_value
    D6_value = D6(delta, X)
    D_5_value = D5(delta, X)
    C32_value = C3(T2, alpha, beta, a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), printing-1)
    C32_times_D6 = C32_value * D6_value
    tilde_C5_value = ((C31_times_D4) ** exponent) * ((C32_times_D6) ** exponent2)
    return tilde_C5_value

tilde_C5(sigma_prime, alpha, beta, delta, delta_2, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2),
              a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), T2,2)

C3_value is 5.473783347188228
C3_value is 42.38082568387099


4.067705422614315

#DEFINITION OF \breve{C_5}

In [104]:
exponent2upper = 2 * (sigma - sigma_1)
result2breve = (C32_times_D6) ** exponent2upper
print(f"The result for sigma2: {result2breve}")

The result for sigma2: 4.240355499784314


In [105]:
def breve_C5(sigma,alpha, beta, delta_2, a, a_i, b, b_i, d_i,
              a_prime, a_i_prime, b_prime, b_i_prime, d_i_prime, T2, printing=0):
    X = T2
    sigma_2 = 1 + (delta / np.log(X))
    sigma_1 = 0.5
    sigma_prime = sigma - (delta_2 / np.log(T2))
    exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
    exponent2upper = 2 * (sigma - sigma_1)

    D4_value = D4(X)
    C31_value = C3(T2, alpha, beta, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2),printing-1)
    C31_times_D4 = C31_value * D4_value
    D6_value = D6(delta, X)
    D_5_value = D5(delta, X)
    C32_value = C3(T2, alpha, beta, a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta),printing-1)
    C32_times_D6 = C32_value * D6_value
    breve_C5_value = ((C31_times_D4) ** exponent) * ((C32_times_D6) ** exponent2upper)
    return breve_C5_value

breve_C5(sigma_prime, alpha, beta, delta_2, a, a_i_values, b, b_i_values, compute_d_i,
              a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), T2,2)

C3_value is 5.473783347188228
C3_value is 42.38082568387099


4.394025740841114

#DEFINITION OF \hat{C5}:

In [106]:
sigma_1_prime = 0.9
sigma_2_prime = 0.9

In [107]:
def hatC5(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, a, a_i, b, b_i, d_i,
              a_prime, a_i_prime, b_prime, b_i_prime, d_i_prime, T2,printing=0):
    """
    Function to compute hat_C5 based on the conditional logic provided:
    If A2 >= A1, it computes bre_C5(sigma_prime_2, T0, delta1, delta2).
    Otherwise, it computes tilde_C5(sigma_prime_1, T0, delta1, delta2).
    """
    X=T2
    sigma_2 = 1 + (delta / np.log(X))
    sigma_1 = 0.5

    D4_value = D4(X)
    C31_value = C3(T2, alpha, beta, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2), printing-1)
    C31_times_D4 = C31_value * D4_value
    D6_value = D6(delta, X)
    D_5_value = D5(delta, X)
    C32_value = C3(T2, alpha, beta, a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), printing-1)
    C32_times_D6 = C32_value * D6_value

    if C32_times_D6 >= C31_times_D4:
        sigma_prime = sigma_2_prime - (delta_2 / np.log(T2))
        exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
        exponent2upper = 2 * (sigma_2_prime - sigma_1)
        breve_C5_value = ((C31_times_D4) ** exponent) * ((C32_times_D6) ** exponent2upper)
        return breve_C5_value
    else:
        sigma_prime = sigma_1_prime - (delta_2 / np.log(T2))
        exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
        exponent2 = (sigma_prime - sigma_1) / (sigma_2 - sigma_1)
        tilde_C5_value = ((C31_times_D4) ** exponent) * ((C32_times_D6) ** exponent2)
        return tilde_C5_value



hatC5_value = hatC5(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2),
              a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), T2,2)

print(f"The result for hatC5 is: {hatC5_value}")

C3_value is 5.473783347188228
C3_value is 42.38082568387099
The result for hatC5 is: 4.5592632151091665


# Define C1 as in equation 4.18

In [108]:
def C1(sigma, T2, alpha, beta):
      C1_value = np.exp(alpha * (sigma / T2)**beta)
      return C1_value
C1_value1 = C1(sigma_1, T2, alpha, beta)
C1_value2 = C1(sigma_2, T2, alpha, beta)
print(f"The result for C1 on half line is: {C1_value1}")
print(f"The result for C1 on sigma2 is: {C1_value2}")
C1_value1 ** exponent
C1_value2 ** exponent2

The result for C1 on half line is: 1.0
The result for C1 on sigma2 is: 1.0


1.0

# Define C2 as in equation 4.18

In [109]:
T1 = 3 * 1e12
def C2(sigma, T1, T2, alpha):
      C2_value = ((1 - (1 / T1))**2) * np.exp((alpha * (sigma / T2)**2) - alpha)
      return C2_value
C2_value = C2(sigma_prime, T1, T2, alpha)
print(f"The result for C2 is: {C2_value}")

The result for C2 is: 0.7858416263878217


#DEFINITION OF C4

In [110]:
# Define C4 as in equation 4.84
def C4(sigma, T2, T1, alpha, beta, delta, delta_2,printing=0):
    X=T2
    sigma_2 = 1 + (delta / np.log(X))
    sigma_1 = 0.5
    sigma_prime = sigma - (delta_2 / np.log(T2))
    exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
    exponent2 = (sigma_prime - sigma_1) / (sigma_2 - sigma_1)

    C2_value = C2(sigma_prime, T1, T2, alpha)
    C1_value1 = C1(sigma_1, T2, alpha, beta)
    C1_value2 = C1(sigma_2, T2, alpha, beta)

    C4_value =  (alpha * beta * (C1_value1**exponent) *  (C1_value2**exponent2)) / C2_value
    return C4_value


C4_value = C4(sigma_prime, T2, T1, alpha, beta, delta, delta_2,2)
print(f"The result for C4 is: {C4_value}")

The result for C4 is: 0.6133551390189752


#DEFINITION OF \HAT{C4}:(MAX OF C4):

In [111]:
def hatC4(sigma_1_prime, sigma_2_prime,T2, T1, alpha, beta,  delta, delta_2,printing=0):
    hatC4_value = ((1 - (1 / T1))**-(2)) * alpha * beta * np.exp(alpha) * np.exp((alpha/(4*T2)) * (sigma_1_prime + sigma_2_prime)**2)
    return hatC4_value

hatC4_value = hatC4(sigma_1_prime, sigma_2_prime, T2, T1, alpha, beta, delta, delta_2,2)
print(f"The result for hatC4 is: {hatC4_value}")

The result for hatC4 is: 0.6133551390190151


#DEFINITION OF C6:

In [112]:
# Define C6 as in equation 4.84
def C6(sigma, T2, a, a_prime, b, b_prime, delta, delta_2,printing=0):

    sigma_prime = sigma - (delta_2 / np.log(T2))
    sigma_2 = 1 + (delta / np.log(T2))
    sigma_1 = 0.5
    exponent = (sigma_2 - sigma_prime) / (sigma_2 - sigma_1)
    exponent2 = (sigma_prime - sigma_1) / (sigma_2 - sigma_1)

    numerator = (
        T2 ** ((a * exponent) + ((a_prime - 1) * exponent2)) *
        (np.log(T2) ** (((b + 1) * exponent) + ((b_prime + 2) * exponent2)))
    )

    # Denominator
    denominator = (
        T2 ** ((2 * a * (1 - sigma)) + ((a_prime - 1) * (2 * sigma - 1))) *
        (np.log(T2) ** ((2 * (b + 1) * (1 - sigma)) + ((b_prime + 2) * (2 * sigma - 1))))
    )
    C6_value =  numerator / denominator
    return C6_value


C6_value = C6(sigma_prime, T2, a, a_prime, b, b_prime, delta, delta_2,2)
print(f"The result for C6 is: {C6_value}")

The result for C6 is: 5.932302463592068


In [113]:
def breveC6(sigma_prime, T2, a, a_prime, delta, delta_2,printing=0):
    sigma_prime = sigma - (delta_2 / np.log(T2))
    breveC6_value = np.exp((1+a-a_prime) * (2 * delta * (2* sigma_prime -1) + 2 * delta_2))
    return breveC6_value


breveC6_value = breveC6(sigma_prime, T2, a, a_prime, delta, delta_2,2)
print(f"The result for breveC6 is: {breveC6_value}")

The result for breveC6 is: 5.354529565142544


#DEFINITION OF HAT_C6

In [114]:
def hatC6(sigma_1_prime, sigma_2_prime, a, a_prime, b, b_prime, delta, delta_2, T2,printing=0):
    """
    Function to compute hat_C6 based on the conditional logic provided:
    If b-b_prime > 1, it computes C6(sigma_2_prime, T2, a, a_prime, b, b_prime).
    Otherwise, it computes breveC6(sigma_1_prime, T2, a, a_prime, delta, delta_2).

    C6_value = C6(sigma_prime, T2, a, a_prime, b, b_prime)
    breveC6_value = breveC6(sigma_1_prime, T2, a, a_prime, delta, delta_2)
    Returns:
         Value of \(\hat{C_6}\).
    """
    if b-b_prime > 1:
        return  C6(sigma_2_prime, T2, a, a_prime, b, b_prime, delta, delta_2,printing-1)
    else:
        return breveC6(sigma_2_prime, T2, a, a_prime, delta, delta_2)
hatC6_value = hatC6(sigma_1_prime, sigma_2_prime, a, a_prime, b, b_prime,delta, delta_2, T2,2)
print(f"The result for hatC6 is: {hatC6_value}")

The result for hatC6 is: 6.081378015582197


In [115]:
def U(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, T2, a, a_i_values, b, b_i_values, compute_d_i, D1, D2,
              a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second, printing=0):
    hatC4_value = hatC4(sigma_1_prime, sigma_2_prime, T2, T1, alpha, beta,delta, delta_2, printing-1)
    hatC5_value = hatC5(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, a, a_i_values, b, b_i_values, compute_d_i(X, D1, D2),
              a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second(X, D5, D6, delta), T2,printing-1)
    hatC6_value = hatC6(sigma_1_prime, sigma_2_prime, a, a_prime, b, b_prime,delta, delta_2, T2,printing-1)
    U_value = ((hatC4_value * hatC5_value * hatC6_value)/(2 * np.pi * delta_2))
    if printing>0:
        print(f"hatC4_value is: {hatC4_value}")
        print(f"hatC5_value is: {hatC5_value}")
        print(f"hatC6_value is: {hatC6_value}")
    return U_value

U_value = U(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, T2, a, a_i_values, b, b_i_values, compute_d_i,D1, D2,
              a_prime, a_i_prime_values, b_prime, b_i_prime_values,compute_d_i_second,2)
print(f"The result for U is: {U_value}")

hatC4_value is: 0.6133551390190151
hatC5_value is: 4.5592632151091665
hatC6_value is: 6.081378015582197
The result for U is: 7.733227043952999


In [116]:
# It takes three parameters: alpha, delta, and delta_2 and assumed the other parameters are fixed
def U_partial(alpha, delta, delta_2):
    return U(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, T2, a, a_i_values, b, b_i_values, compute_d_i,D1, D2,
              a_prime, a_i_prime_values, b_prime, b_i_prime_values,compute_d_i_second)


lambdaoptX = lambda ADD: U_partial(ADD[0], ADD[1], ADD[2])

# Initial guess for [alpha, delta, delta_2]
initial_guess = [0.15, 0.33, 0.33]


bounds = [(0, 1), (0, 1), (0, 1)]

result = minimize(lambdaoptX, initial_guess, bounds=bounds, method='L-BFGS-B')

optimal_params = result.x
optimal_U = result.fun

print(f"Optimal parameters: alpha={optimal_params[0]}, delta={optimal_params[1]}, delta_2={optimal_params[2]}")
print(f"Minimum value of U: {optimal_U}")

Optimal parameters: alpha=0.2419871507835905, delta=0.36641375030212875, delta_2=0.3511175514777831
Minimum value of U: 7.732290624302065


In [117]:
# Define sigma_1' and sigma_2' ranges
sigma_1_prime_values = np.arange(0.75, 0.99, 0.02)
sigma_2_prime_values = np.arange(0.77, 0.99, 0.02)

results = []

min_length = min(len(sigma_1_prime_values), len(sigma_2_prime_values))

for i in range(min_length):
    sigma_1_prime = sigma_1_prime_values[i]
    sigma_2_prime = sigma_2_prime_values[i]

    def U_partial(alpha, delta, delta_2):
        return U(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, T2, a, a_i_values, b, b_i_values, compute_d_i, D1, D2,
                  a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second)

    lambdaoptX = lambda ADD: U_partial(ADD[0], ADD[1], ADD[2])

    initial_guess = [0.15, 0.33, 0.33]
    bounds = [(0, 1), (0, 1), (0.1, 1)]

    result = minimize(lambdaoptX, initial_guess, bounds=bounds, method='L-BFGS-B')

    optimal_params = result.x
    optimal_U = result.fun
    results.append([sigma_1_prime, sigma_2_prime, optimal_params[0], optimal_params[1], optimal_params[2], optimal_U])

columns = ["Sigma_1'", "Sigma_2'", "Alpha", "Delta", "Delta_2", "Min U"]
df_results = pd.DataFrame(results, columns=columns)

print(df_results)

    Sigma_1'  Sigma_2'     Alpha     Delta   Delta_2      Min U
0       0.75      0.77  0.386811  0.379884  0.358105   3.848096
1       0.77      0.79  0.364310  0.377590  0.357185   4.303122
2       0.79      0.81  0.341859  0.375429  0.356220   4.804979
3       0.81      0.83  0.319477  0.373362  0.355203   5.357106
4       0.83      0.85  0.297181  0.371355  0.354126   5.962880
5       0.85      0.87  0.274994  0.369378  0.352982   6.625508
6       0.87      0.89  0.252947  0.367406  0.351761   7.347899
7       0.89      0.91  0.231077  0.365414  0.350450   8.132489
8       0.91      0.93  0.209432  0.363378  0.349037   8.981032
9       0.93      0.95  0.188078  0.361274  0.347508   9.894318
10      0.95      0.97  0.167096  0.359076  0.345841  10.871848


In [118]:
latex_table = df_results.to_latex(index=False, escape=False)

print(latex_table)

\begin{tabular}{rrrrrr}
\toprule
Sigma_1' & Sigma_2' & Alpha & Delta & Delta_2 & Min U \\
\midrule
0.750000 & 0.770000 & 0.386811 & 0.379884 & 0.358105 & 3.848096 \\
0.770000 & 0.790000 & 0.364310 & 0.377590 & 0.357185 & 4.303122 \\
0.790000 & 0.810000 & 0.341859 & 0.375429 & 0.356220 & 4.804979 \\
0.810000 & 0.830000 & 0.319477 & 0.373362 & 0.355203 & 5.357106 \\
0.830000 & 0.850000 & 0.297181 & 0.371355 & 0.354126 & 5.962880 \\
0.850000 & 0.870000 & 0.274994 & 0.369378 & 0.352982 & 6.625508 \\
0.870000 & 0.890000 & 0.252947 & 0.367406 & 0.351761 & 7.347899 \\
0.890000 & 0.910000 & 0.231077 & 0.365414 & 0.350450 & 8.132489 \\
0.910000 & 0.930000 & 0.209432 & 0.363378 & 0.349037 & 8.981032 \\
0.930000 & 0.950000 & 0.188078 & 0.361274 & 0.347508 & 9.894318 \\
0.950000 & 0.970000 & 0.167096 & 0.359076 & 0.345841 & 10.871848 \\
\bottomrule
\end{tabular}



In [119]:
sigma_values = np.linspace(0.75, 0.99, 20)

results = []

for sigma in sigma_values:
    sigma_1_prime = sigma_2_prime = sigma  #just for one point

    def U_partial(alpha, delta, delta_2):
        return U(sigma_1_prime, sigma_2_prime, alpha, beta, delta, delta_2, T2, a, a_i_values, b, b_i_values, compute_d_i, D1, D2,
                  a_prime, a_i_prime_values, b_prime, b_i_prime_values, compute_d_i_second)

    lambdaoptX = lambda ADD: U_partial(ADD[0], ADD[1], ADD[2])

    initial_guess = [0.15, 0.33, 0.33]
    bounds = [(0, 1), (0, 1), (0.1, 1)]

    result = minimize(lambdaoptX, initial_guess, bounds=bounds, method='L-BFGS-B')

    optimal_params = result.x
    optimal_U = result.fun
    results.append([sigma, optimal_params[0], optimal_params[1], optimal_params[2], optimal_U])

columns = ["Sigma", "Alpha", "Delta", "Delta_2", "Min U"]
df_results = pd.DataFrame(results, columns=columns)

print(df_results)

<ipython-input-92-6594859c24f2>:27: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, 0, np.inf)
<ipython-input-92-6594859c24f2>:27: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  result, _ = quad(integrand, 0, np.inf)
<ipython-input-92-6594859c24f2>:27: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result, 

       Sigma     Alpha     Delta   Delta_2      Min U
0   0.750000  0.409355  0.382366  0.358985   3.436473
1   0.762632  0.395113  0.380774  0.358433   3.691576
2   0.775263  0.380886  0.379264  0.357867   3.963493
3   0.787895  0.366676  0.377824  0.357284   4.253079
4   0.800526  0.352486  0.376438  0.356682   4.561201
5   0.813158  0.338321  0.375097  0.356064   4.888727
6   0.825789  0.324182  0.373792  0.355422   5.236522
7   0.838421  0.310077  0.372512  0.354757   5.605440
8   0.851053  0.296010  0.371250  0.354068   5.996311
9   0.863684  0.281987  0.370000  0.353351   6.409931
10  0.876316  0.268016  0.368756  0.352606   6.847048
11  0.888947  0.254104  0.367510  0.351827   7.308346
12  0.901579  0.240261  0.366256  0.351014   7.794423
13  0.914211  0.226500  0.364988  0.350161   8.305772
14  0.926842  0.212833  0.363703  0.349268   8.842754
15  0.939474  0.199276  0.362392  0.348329   9.405566
16  0.952105  0.185850  0.361047  0.347339   9.994213
17  0.964737  0.172576  0.35

In [120]:
latex_table = df_results.to_latex(index=False, escape=False)

print(latex_table)

\begin{tabular}{rrrrr}
\toprule
Sigma & Alpha & Delta & Delta_2 & Min U \\
\midrule
0.750000 & 0.409355 & 0.382366 & 0.358985 & 3.436473 \\
0.762632 & 0.395113 & 0.380774 & 0.358433 & 3.691576 \\
0.775263 & 0.380886 & 0.379264 & 0.357867 & 3.963493 \\
0.787895 & 0.366676 & 0.377824 & 0.357284 & 4.253079 \\
0.800526 & 0.352486 & 0.376438 & 0.356682 & 4.561201 \\
0.813158 & 0.338321 & 0.375097 & 0.356064 & 4.888727 \\
0.825789 & 0.324182 & 0.373792 & 0.355422 & 5.236522 \\
0.838421 & 0.310077 & 0.372512 & 0.354757 & 5.605440 \\
0.851053 & 0.296010 & 0.371250 & 0.354068 & 5.996311 \\
0.863684 & 0.281987 & 0.370000 & 0.353351 & 6.409931 \\
0.876316 & 0.268016 & 0.368756 & 0.352606 & 6.847048 \\
0.888947 & 0.254104 & 0.367510 & 0.351827 & 7.308346 \\
0.901579 & 0.240261 & 0.366256 & 0.351014 & 7.794423 \\
0.914211 & 0.226500 & 0.364988 & 0.350161 & 8.305772 \\
0.926842 & 0.212833 & 0.363703 & 0.349268 & 8.842754 \\
0.939474 & 0.199276 & 0.362392 & 0.348329 & 9.405566 \\
0.952105 & 0.185850 